In [97]:
import ROOT
import numpy as np
import pandas as pd
h   = 6.62607015e-34  # Planck's constant [J*s]
m_e = 9.10938356e-31  # Electron rest mass [kg]
q   = 1.60217663e-19  # Elementary charge [C]

def fit_lin(x, sx, y, sy, name="fit", visual_scale_factor_x=1, visual_scale_factor_y=1):
    ROOT.gStyle.SetOptFit(1)
    graph = ROOT.TGraphErrors(len(x))
    for i in range(len(x)):
        graph.SetPoint(i, x[i], y[i])
        graph.SetPointError(i, sx[i], sy[i])

    fit_func = ROOT.TF1("fit_func", "[0] + [1]*x", min(x), max(x))
    fit_func.SetParameters(0, 1)  # initial guesses

    graph.Fit(fit_func)
    cal_parameters=[(fit_func.GetParameter(i),fit_func.GetParError(i)) for i in range(2)]

    starting_point=0
    if visual_scale_factor_x != 1 or visual_scale_factor_y !=1:
        print(f"Applying a visual scaling factor of {visual_scale_factor_x} to x and {visual_scale_factor_y} to x and from data points{starting_point+1}.")
        for i in range(starting_point,len(x)):
            graph.SetPointError(i, sx[i] * visual_scale_factor_x, sy[i] * visual_scale_factor_y)
    graph.SetMarkerStyle(20)
    graph.SetMarkerColor(ROOT.kBlue)
    graph.SetTitle(name)

    c = ROOT.TCanvas("c", "Line Fit", 800, 600)
    graph.Draw("AP")
    fit_func.Draw("same")
    c.Update()
    stats = graph.GetListOfFunctions().FindObject("stats")
    if stats:
        stats.SetX1NDC(0.15)  # lower-left x (0 to 1)
        stats.SetY1NDC(0.75)  # lower-left y
        stats.SetX2NDC(0.45)  # upper-right x
        stats.SetY2NDC(0.87)   # upper-right y
        stats.SetTextSize(0.03)
        c.Modified()

    ROOT.gPad.SetTicks(1, 1)
    ROOT.gPad.SetGrid(1, 1)
    c.Update()
    c.SaveAs(f"{name}.pdf")
    del c, graph, fit_func
    return cal_parameters


first let us calcualte debrogie wavelength from: $$\lambda=\frac{h}{\sqrt{2q_em_eV}}$$  $$\sigma_{\lambda}=\frac{h\sigma_V}{\sqrt{8q_em_eV^3}}=\lambda\frac{\sigma_V}{2V}$$

In [98]:
def get_lambda(V,V_s=None):

    coeff_meters = h / np.sqrt(2 * m_e * q)

    coeff_nm = coeff_meters * 1.0e9
    lambda_nm=[]
    for i in V:
        lambda_nm.append(float(coeff_nm / (np.sqrt(i))))

    if V_s is not None:
        d_uncertainty_nm=[]
        for i in range(len(V_s)):
            d_uncertainty_nm.append(float(lambda_nm[i] * (V_s[i] /(2*V[i]))))
        return lambda_nm,d_uncertainty_nm

    return lambda_nm

and anel calculaton from arch length s follows: $$r = R \sin\left(\frac{s}{2R}\right)$$  $$\theta = \frac{1}{2} \arctan\left(\frac{r}{L}\right)$$  $$\sigma_r = \sqrt{ \left[ \frac{1}{2} \cos\left(\frac{s}{2R}\right) \sigma_s \right]^2 + \left[ \left( \sin\left(\frac{s}{2R}\right) - \frac{s}{2R}\cos\left(\frac{s}{2R}\right) \right) \sigma_R \right]^2 }= \frac{1}{2} \cos^2(2\theta) \sqrt{ \left( \frac{\sigma_r}{L} \right)^2 + \left( \frac{r \sigma_L}{L^2} \right)^2 }$$

In [99]:


def calculate_theta(s_arc, s_err, L=14.0, L_err=0.3, R=4.3, R_err=0.1):
    """
    Vectorized version: Converts measured arc lengths to theta.
    Accepts single values OR lists/pandas Series.
    """
    s_arc = np.array(s_arc)
    s_err = np.array(s_err)
 
    # r = R * sin(s / (2*R))
    term = s_arc / (2 * R)
    r = R * np.sin(term)
    
    # Derivatives for r
    dr_ds = 0.5 * np.cos(term)
    dr_dR = np.sin(term) - term * np.cos(term)
    
    # Error in r
    r_err = np.sqrt((dr_ds * s_err)**2 + (dr_dR * R_err)**2)

    # 2*theta = arctan(r / L)
    u = r / L
    two_theta = np.arctan(u)
    
    # Derivatives for 2*theta
    denom = 1 + u**2
    d2theta_dr = (1/L) / denom
    d2theta_dL = (-r/L**2) / denom
    
    # Error in 2*theta
    two_theta_err = np.sqrt((d2theta_dr * r_err)**2 + (d2theta_dL * L_err)**2)
    
    theta_rad = two_theta / 2
    theta_err = two_theta_err / 2
    
    return theta_rad, theta_err

In [100]:
# Load the dataset
df = pd.read_csv('Diffraction_Graphite.csv')
# accessing the first column 'Voltage (V)' and taking every 2nd value
voltage_list = df['Voltage (V)'].tolist() #[::2]
print("Voltage Values:", voltage_list)
voltage_s_list=np.full(len(voltage_list), 100)
print(voltage_s_list)


Voltage Values: [2900, 2900, 3200, 3200, 3500, 3500, 3800, 3800, 3900, 3900, 4000, 4000, 4100, 4100, 4300, 4300, 4600, 4600, 5000, 5000]
[100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100]


In [101]:
df = pd.read_csv('Diffraction_Graphite.csv')
diams = df[['Inside Diameter (cm)', 'Outside Diameter (cm)', 'Inside Horizontal(cm)', 'Outside Horizontal (cm)']]
voltage_list = df['Voltage (V)'].tolist()
print("Voltage Values:", voltage_list)
voltage_s_list=np.full(len(voltage_list), 100)
df["lambda_nm"], df["lambda_unc_nm"] = get_lambda(voltage_list,voltage_s_list)
radii_df = diams / 2.0
df['Radius_cm'] = radii_df.mean(axis=1)

# ddof=1 ensures we calculate Sample Standard Deviation (N-1)
df['Radius_unc_cm'] = radii_df.std(axis=1, ddof=1)
df["theta_val_rad"],df["theta_unc_rad"]=calculate_theta(df['Radius_cm'], df['Radius_unc_cm'])
print(df[['Voltage (V)', 'Ring Order', 'Radius_cm', 'Radius_unc_cm','lambda_nm','lambda_unc_nm','theta_val_rad','theta_unc_rad']])

Voltage Values: [2900, 2900, 3200, 3200, 3500, 3500, 3800, 3800, 3900, 3900, 4000, 4000, 4100, 4100, 4300, 4300, 4600, 4600, 5000, 5000]
    Voltage (V)  Ring Order  Radius_cm  Radius_unc_cm  lambda_nm  \
0          2900   1st Order      0.850       0.057735   0.022774   
1          2900   2nd Order      1.400       0.057735   0.022774   
2          3200   1st Order      0.725       0.064550   0.021680   
3          3200   2nd Order      1.275       0.104083   0.021680   
4          3500   1st Order      0.700       0.057735   0.020730   
5          3500   2nd Order      1.225       0.064550   0.020730   
6          3800   1st Order      0.675       0.064550   0.019895   
7          3800   2nd Order      1.175       0.064550   0.019895   
8          3900   1st Order      0.650       0.057735   0.019639   
9          3900   2nd Order      1.175       0.064550   0.019639   
10         4000   1st Order      0.675       0.064550   0.019391   
11         4000   2nd Order      1.200       0.

In [102]:
df

,Voltage (V),Ring Order,Inside Diameter (cm),Outside Diameter (cm),Inside Horizontal(cm),Outside Horizontal (cm),Filtering Voltage (A),lambda_nm,lambda_unc_nm,Radius_cm,Radius_unc_cm,theta_val_rad,theta_unc_rad
0,2900,1st Order,1.6,1.8,1.6,1.8,0.42,0.022774,0.000393,0.850,0.057735,0.015149,0.001075
1,2900,2nd Order,2.7,2.9,2.7,2.9,0.42,0.022774,0.000393,1.400,0.057735,0.024869,0.001146
2,3200,1st Order,1.4,1.6,1.3,1.5,0.42,0.021680,0.000339,0.725,0.064550,0.012928,0.001181
3,3200,2nd Order,2.6,2.8,2.3,2.5,0.42,0.021680,0.000339,1.275,0.104083,0.022669,0.001898
4,3500,1st Order,1.3,1.5,1.3,1.5,0.42,0.020730,0.000296,0.700,0.057735,0.012484,0.001061
5,3500,2nd Order,2.4,2.6,2.3,2.5,0.42,0.020730,0.000296,1.225,0.064550,0.021787,0.001231
6,3800,1st Order,1.3,1.5,1.2,1.4,0.42,0.019895,0.000262,0.675,0.064550,0.012039,0.001177
7,3800,2nd Order,2.3,2.5,2.2,2.4,0.42,0.019895,0.000262,1.175,0.064550,0.020905,0.001225
8,3900,1st Order,1.2,1.4,1.2,1.4,0.42,0.019639,0.000252,0.650,0.057735,0.011594,0.001057
9,3900,2nd Order,2.3,2.5,2.2,2.4,0.42,0.019639,0.000252,1.175,0.064550,0.020905,0.001225


In [103]:
df.to_csv('processed_data.csv', index=False,float_format='%.4e')

we will fit $1/\sqrt{V}$ vs. $sin(\theta)$:
\begin{equation}
    2d\sin(\theta)=n\lambda=1\left(\frac{h}{\sqrt{2m_eq_eV}}\right)
\end{equation}    

\begin{equation}
    \underbrace{\sin(\theta)}_{y} = \underbrace{\left[ \frac{h}{2d\sqrt{2m_e q}} \right]}_{\text{Slope } m} \cdot \underbrace{\frac{1}{\sqrt{V}}}_{x}
\end{equation}

In [104]:
# 1. Load the data
df = pd.read_csv('processed_data.csv')

# 2. Clean the 'Ring Order' column (fix "2nd  Order" typo)
df['Ring Order'] = df['Ring Order'].str.replace('  ', ' ', regex=False)

# 3. Create separate DataFrames
df_1st = df[df['Ring Order'] == '1st Order'].copy()
df_2nd = df[df['Ring Order'] == '2nd Order'].copy()

# 5. Display to verify
df_1st.to_csv('processed_data1.csv', index=False,float_format='%.4e')
df_2nd.to_csv('processed_data2.csv', index=False,float_format='%.4e')

In [105]:
df_1st

,Voltage (V),Ring Order,Inside Diameter (cm),Outside Diameter (cm),Inside Horizontal(cm),Outside Horizontal (cm),Filtering Voltage (A),lambda_nm,lambda_unc_nm,Radius_cm,Radius_unc_cm,theta_val_rad,theta_unc_rad
0,2900,1st Order,1.6,1.8,1.6,1.8,0.42,0.022774,0.000393,0.850,0.057735,0.015149,0.001075
2,3200,1st Order,1.4,1.6,1.3,1.5,0.42,0.021680,0.000339,0.725,0.064550,0.012928,0.001181
4,3500,1st Order,1.3,1.5,1.3,1.5,0.42,0.020730,0.000296,0.700,0.057735,0.012484,0.001061
6,3800,1st Order,1.3,1.5,1.2,1.4,0.42,0.019895,0.000262,0.675,0.064550,0.012039,0.001177
8,3900,1st Order,1.2,1.4,1.2,1.4,0.42,0.019639,0.000252,0.650,0.057735,0.011594,0.001057
10,4000,1st Order,1.3,1.5,1.2,1.4,0.42,0.019391,0.000242,0.675,0.064550,0.012039,0.001177
12,4100,1st Order,1.2,1.4,1.2,1.4,0.42,0.019154,0.000234,0.650,0.057735,0.011594,0.001057
14,4300,1st Order,1.2,1.4,1.1,1.3,0.42,0.018703,0.000217,0.625,0.064550,0.011149,0.001174
16,4600,1st Order,1.2,1.4,1.1,1.3,0.42,0.018083,0.000197,0.625,0.064550,0.011149,0.001174
18,5000,1st Order,1.1,1.3,1.1,1.3,0.42,0.017344,0.000173,0.600,0.057735,0.010704,0.001053


In [106]:
df_2nd

,Voltage (V),Ring Order,Inside Diameter (cm),Outside Diameter (cm),Inside Horizontal(cm),Outside Horizontal (cm),Filtering Voltage (A),lambda_nm,lambda_unc_nm,Radius_cm,Radius_unc_cm,theta_val_rad,theta_unc_rad
1,2900,2nd Order,2.7,2.9,2.7,2.9,0.42,0.022774,0.000393,1.400,0.057735,0.024869,0.001146
3,3200,2nd Order,2.6,2.8,2.3,2.5,0.42,0.021680,0.000339,1.275,0.104080,0.022669,0.001898
5,3500,2nd Order,2.4,2.6,2.3,2.5,0.42,0.020730,0.000296,1.225,0.064550,0.021787,0.001231
7,3800,2nd Order,2.3,2.5,2.2,2.4,0.42,0.019895,0.000262,1.175,0.064550,0.020905,0.001225
9,3900,2nd Order,2.3,2.5,2.2,2.4,0.42,0.019639,0.000252,1.175,0.064550,0.020905,0.001225
11,4000,2nd Order,2.3,2.5,2.3,2.5,0.42,0.019391,0.000242,1.200,0.057735,0.021346,0.001117
13,4100,2nd Order,2.2,2.4,2.1,2.3,0.42,0.019154,0.000234,1.125,0.064550,0.020021,0.001219
15,4300,2nd Order,2.2,2.4,2.1,2.3,0.42,0.018703,0.000217,1.125,0.064550,0.020021,0.001219
17,4600,2nd Order,2.2,2.4,2.1,2.3,0.42,0.018083,0.000197,1.125,0.064550,0.020021,0.001219
19,5000,2nd Order,2.0,2.2,2.0,2.2,0.42,0.017344,0.000173,1.050,0.057735,0.018695,0.001097


In [ ]:
sigma_V = 100.0 

# --- 2. Helper Function to Prepare Data ---
def prepare_fit_data(df, sigma_V):
    """
    Extracts and transforms dataframe columns into vectors for the fit.
    X = 1/sqrt(V), Y = sin(theta)
    """
    # Get raw arrays
    V = df['Voltage (V)'].to_numpy()
    theta = df['theta_val_rad'].to_numpy()
    theta_err = df['theta_unc_rad'].to_numpy()
    
    # --- X Axis Calculations: 1/sqrt(V) ---
    x = 1.0 / np.sqrt(V)
    
    # Error Propagation for x = V^(-0.5)
    # sigma_x = |dx/dV| * sigma_V = |-0.5 * V^(-1.5)| * sigma_V
    # Easier form: sigma_x = x * 0.5 * (sigma_V / V)
    sx = x * 0.5 * (sigma_V / V)
    
    # --- Y Axis Calculations: sin(theta) ---
    y = np.sin(theta)
    
    # Error Propagation for y = sin(theta)
    # sigma_y = |dy/dtheta| * sigma_theta = cos(theta) * sigma_theta
    sy = np.cos(theta) * theta_err
    
    return x, sx, y, sy

# --- 3. Prepare Data for Both Orders ---
x_1st, sx_1st, y_1st, sy_1st = prepare_fit_data(df_1st, sigma_V)
x_2nd, sx_2nd, y_2nd, sy_2nd = prepare_fit_data(df_2nd, sigma_V)

print("--- Fitting 1st Order ---")
params_1st = fit_lin(x_1st, sx_1st, y_1st, sy_1st, name="1st Order V vs. sin(theta)")
print("\nFit Results (Intercept, Slope):")
print(f"1st Order: {params_1st}")
print("\n--- Fitting 2nd Order ---")
params_2nd = fit_lin(x_2nd, sx_2nd, y_2nd, sy_2nd, name="2nd Order V vs. sin(theta)")

print("\nFit Results (Intercept, Slope):")
print(f"2nd Order: {params_2nd}")

--- Fitting 1st Order ---

Fit Results (Intercept, Slope):
1st Order: [(-0.0024731533492888856, 0.004488756414430641), (0.9046897423568036, 0.2782457989467004)]

--- Fitting 2nd Order ---

Fit Results (Intercept, Slope):
2nd Order: [(0.0005735323326288497, 0.005000290621989732), (1.2795984341290896, 0.3123853066502074)]
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1.29613
NDf                       =            8
Edm                       =  3.18156e-07
NCalls                    =           49
p0                        =  -0.00247315   +/-   0.00448876  
p1                        =      0.90469   +/-   0.278246    
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1.13827
NDf                       =            8
Edm                       =  1.64619e-05
NCalls                    =           45
p0                        =  0.000573532   +/-   0.00500029  
p1                    

Info in <TCanvas::Print>: pdf file 1st Order V vs. sin(theta).pdf has been created
Info in <TCanvas::Print>: pdf file 2nd Order V vs. sin(theta).pdf has been created


$$slope=\frac{h}{2d\sqrt{2m_e q}}$$
then latice constant d: 
$$d=\frac{h}{2\cdot slope\sqrt{2m_e q}}$$
and the uncertainty is:
$$\sigma_d=d \cdot \frac{slope}{\sigma_{slope}}

In [108]:
def get_lattice_spacing(slope, slope_err=None):
    """
    Calculates lattice spacing d from the slope of sin(theta) vs 1/sqrt(V).
    
    Parameters:
    slope     : The slope m obtained from the linear fit [units: sqrt(V)]
    slope_err : The uncertainty in the slope from the fit (sigma_m)
    
    Returns:
    d_nm      : Lattice spacing in nanometers
    d_err_nm  : Uncertainty in lattice spacing in nanometers (if slope_err provided)
    """
    
    # 1. Define Constants
    h   = 6.62607015e-34  # Planck's constant [J*s]
    m_e = 9.10938356e-31  # Electron rest mass [kg]
    q   = 1.60217663e-19  # Elementary charge [C]
    
    # 2. Calculate the theoretical coefficient
    # This represents lambda * sqrt(V) = h / sqrt(2*me*q)
    # Value is approx 1.226 nm * V^0.5
    coeff_meters = h / np.sqrt(2 * m_e * q)
    coeff_nm = coeff_meters * 1.0e9
    
    # 3. Calculate d
    # Formula: d = Coeff / (2 * Slope)
    d_nm = coeff_nm / (2 * slope)
    
    # 4. Error Propagation
    # Since d = k / m, relative error is the same: sigma_d/d = sigma_m/m
    # sigma_d = d * (sigma_m / m)
    if slope_err is not None:
        d_err_nm = d_nm * (slope_err / slope)
        return d_nm, d_err_nm
    
    return d_nm

In [113]:
slope,slope_unc=params_1st[1]
d_val, d_unc = get_lattice_spacing(slope, slope_unc)

print(f"Lattice Constant d: {d_val:.4e} ± {d_unc:.4e} nm")
slope,slope_unc=params_2nd[1]
d_val, d_unc = get_lattice_spacing(slope, slope_unc)

print(f"Lattice Constant d: {d_val:.4e} ± {d_unc:.4e} nm")

Lattice Constant d: 6.7782e-01 ± 2.0847e-01 nm
Lattice Constant d: 4.7922e-01 ± 1.1699e-01 nm


In [111]:
def get_slope(slope, slope_err=None):
    """
    Calculates lattice spacing d from the slope of sin(theta) vs 1/sqrt(V).
    
    Parameters:
    slope     : The slope m obtained from the linear fit [units: sqrt(V)]
    slope_err : The uncertainty in the slope from the fit (sigma_m)
    
    Returns:
    d_nm      : Lattice spacing in nanometers
    d_err_nm  : Uncertainty in lattice spacing in nanometers (if slope_err provided)
    """
    
    # 1. Define Constants
    h   = 6.62607015e-34  # Planck's constant [J*s]
    m_e = 9.10938356e-31  # Electron rest mass [kg]
    q   = 1.60217663e-19  # Elementary charge [C]
    
    # 2. Calculate the theoretical coefficient
    # This represents lambda * sqrt(V) = h / sqrt(2*me*q)
    # Value is approx 1.226 nm * V^0.5
    coeff_meters = h / np.sqrt(2 * m_e * q)
    coeff_nm = coeff_meters * 1.0e9
    
    # 3. Calculate d
    # Formula: d = Coeff / (2 * Slope)
    d_nm = coeff_nm / (2 * slope)
    
    # 4. Error Propagation
    # Since d = k / m, relative error is the same: sigma_d/d = sigma_m/m
    # sigma_d = d * (sigma_m / m)
    if slope_err is not None:
        d_err_nm = d_nm * (slope_err / slope)
        return d_nm, d_err_nm
    
    return d_nm

In [112]:
get_slope(0.123)

np.float64(4.985471452091064)